<a href="https://colab.research.google.com/github/ar-yansingh/Capstone-Project/blob/main/Model1_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pathway bokeh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 

In [114]:
import pandas as pd
import pathway as pw
from pathway.internals import this
import time

In [115]:

# Load Dataset with Pandas

df = pd.read_csv("dataset.csv")

In [116]:

# Define Schema for Pathway

class ParkingData(pw.Schema):
    LastUpdatedDate: str
    LastUpdatedTime: str
    Capacity: float
    Occupancy: float
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: float
    IsSpecialDay: int

In [117]:

# Stream Into Pathway Table

table = pw.debug.table_from_pandas(df)

In [103]:
# Combine date and time into a timestamp
table = table.with_columns(
    Timestamp = pw.apply(lambda date, time: f"{date} {time}", table.LastUpdatedDate, table.LastUpdatedTime)
)

# Compute occupancy ratio
table = table.with_columns(
    OccRatio = pw.if_else(table.Capacity > 0, table.Occupancy / table.Capacity, 0)
)

# Map traffic condition to numeric
def traffic_to_num(traffic):
    mapping = {"Low": 0.0, "Medium": 0.5, "High": 1.0}
    return mapping.get(traffic, 0.5)

table = table.with_columns(
    TrafficNum = pw.apply(traffic_to_num, table.TrafficConditionNearby)
)

# Map vehicle type to weight
def vehicle_type_weight(vtype):
    mapping = {"car": 1.0, "bike": 0.7, "truck": 1.3}
    return mapping.get(vtype, 1.0)

table = table.with_columns(
    VehicleWeight = pw.apply(vehicle_type_weight, table.VehicleType)
)


In [104]:
# Set weights (tune as needed)
ALPHA = 1.2    # Occupancy
BETA = 0.7     # Queue
GAMMA = 0.5    # Traffic
DELTA = 0.8  # Special day
EPSILON = 0.5  # Vehicle type

table = table.with_columns(
    DemandScore = pw.apply(
        lambda occ, q, s, v, t: float(ALPHA * occ + BETA * q / 10 + DELTA * s + EPSILON * v - GAMMA * t),
        table.OccRatio, table.QueueLength, table.IsSpecialDay, table.VehicleWeight, table.TrafficNum
    )
)

In [105]:
# Normalize demand for price scaling
def normalize_demand(score):
    return max(0.0, min(2.0, float(score)))  # Clamp to [0, 2]

table = table.with_columns(
    NormDemand = pw.apply(normalize_demand, table.DemandScore)
)

BASE_PRICE = 10
LAMBDA = 0.5  # Controls price sensitivity

table = table.with_columns(
    RawPrice = pw.apply(lambda norm: BASE_PRICE * (1 + LAMBDA * norm), table.NormDemand)
)

# Create boolean columns for the conditions
table = table.with_columns(
    PriceBelowMin = pw.apply(lambda raw_price: float(raw_price) < 5.0, table.RawPrice),
    PriceAboveMax = pw.apply(lambda raw_price: float(raw_price) > 20.0, table.RawPrice)
)

# Use apply with boolean columns to determine the final price
table = table.with_columns(
    Price = pw.apply(
        lambda raw_price, below_min, above_max: 5.0 if below_min else (20.0 if above_max else raw_price),
        table.RawPrice, table.PriceBelowMin, table.PriceAboveMax
    )
)

In [106]:
start_time = time.time()
pw.run()
print(f"✅ Pathway pipeline executed in {time.time() - start_time:.2f} seconds.")


Output()

✅ Pathway pipeline executed in 0.06 seconds.


In [107]:
df_prices = pw.debug.table_to_pandas(
    table.select(
        this.Timestamp,
        this.Price,
        this.VehicleType,
        this.QueueLength,
        this.TrafficConditionNearby,
        this.IsSpecialDay,
        this.SystemCodeNumber,
        this.DemandScore,
        this.Occupancy # Added Occupancy
    )
)

In [120]:


# Convert Timestamp to datetime
df_prices['Timestamp'] = pd.to_datetime(df_prices['Timestamp'], format="%d-%m-%Y %H:%M:%S")

# Sort and reset index for plotting
df_prices = df_prices.sort_values(by='Timestamp').reset_index(drop=True)

# Ensure correct types for all columns used in tooltips
df_prices['Price'] = df_prices['Price'].astype(float)
df_prices['DemandScore'] = df_prices['DemandScore'].astype(float)
df_prices['Occupancy'] = df_prices['Occupancy'].astype(float)
df_prices['QueueLength'] = df_prices['QueueLength'].astype(float)
df_prices['TrafficConditionNearby'] = df_prices['TrafficConditionNearby'].astype(str)
df_prices['IsSpecialDay'] = df_prices['IsSpecialDay'].astype(str)
df_prices['VehicleType'] = df_prices['VehicleType'].astype(str)


In [119]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool

# Make Bokeh output appear directly in the notebook (not in a pop-up window)
output_notebook()

# Convert your DataFrame into a Bokeh data source allows Bokeh to access your data columns by name for plotting and tooltips

source = ColumnDataSource(df_prices)

# Define what information will show up in the tooltip when you hover over a data point
tooltips = [
    ("Time", "@Timestamp{%F %H:%M}"),
    ("Price", "@Price{0.2f}"),
    ("Demand", "@DemandScore{0.2f}"),
    ("Occupancy", "@Occupancy{0}"),
    ("Queue", "@QueueLength{0}"),
    ("Traffic", "@TrafficConditionNearby"),
    ("Special Day", "@IsSpecialDay"),
    ("Vehicle", "@VehicleType"),
]

# Set up the figure for your time series plot
# The x-axis is datetime, and you get interactive tools for zooming and panning
p = figure(
    x_axis_type='datetime',
    title='Dynamic Parking Price Over Time (Model 2)',
    x_axis_label='Time',
    y_axis_label='Price ($)',
    width=950, height=450,
    tools="pan,wheel_zoom,box_zoom,reset"
)

# Draw the main price line
# Thin black line creates a "spiky" effect for dense time series data
p.line('Timestamp', 'Price', source=source, line_width=1, color='black', legend_label="Model 2 Price")

# Add small green circles to highlight each individual price point
p.circle('Timestamp', 'Price', source=source, size=3, color='green', alpha=0.7)

# Add a hover tool so users can see details for each point just by hovering
hover = HoverTool(
    tooltips=tooltips,
    formatters={'@Timestamp': 'datetime'},
    mode='vline'
)
p.add_tools(hover)

# Place the legend in the top left for easy reference
p.legend.location = 'top_left'

# Make grid lines faint for a cleaner look
p.grid.grid_line_alpha = 0.3

# Finally, display the plot in your notebook
show(p)



In [118]:
# Save and load to ensure a clean pandas DataFrame
temp_csv_path = "/tmp/df_prices_temp.csv"
df_prices.to_csv(temp_csv_path, index=False)
df_prices_clean = pd.read_csv(temp_csv_path)

# Make Bokeh plots appear directly in your notebook
output_notebook()

# Convert the 'Timestamp' column to datetime objects for proper time-series plotting
df_prices_clean['Timestamp'] = pd.to_datetime(df_prices_clean['Timestamp'], format="%Y-%m-%d %H:%M:%S")
# Sort the DataFrame by timestamp for correct line plotting
df_prices_clean = df_prices_clean.sort_values(by='Timestamp').reset_index(drop=True)

# Get a list of all unique parking lot identifiers
lots = df_prices_clean['SystemCodeNumber'].unique()

# Set up the Bokeh figure for comparing prices across lots
p = figure(
    x_axis_type='datetime',
    title='Price Comparison Across Lots',
    x_axis_label='Time',
    y_axis_label='Price',
    width=950,
    height=450,
    tools="pan,wheel_zoom,box_zoom,reset"
)

# Loop through each lot and plot its price trend as a separate colored line
for i, lot in enumerate(lots):

    lot_df = df_prices_clean[df_prices_clean['SystemCodeNumber'] == lot].copy()
    # Create a Bokeh data source for this lot
    source = ColumnDataSource(lot_df)
    # Plot the price line for this lot, using a unique color and legend label
    p.line(
        'Timestamp', 'Price',
        source=source,
        color=Category10[10][i % 10],
        legend_label=f"Lot {lot}"
    )


p.legend.click_policy = "hide"


show(p)